In [81]:
import geopandas as gpd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [82]:
# Load the GeoJSON file
gdf = gpd.read_file("Dentists.geojson")
numeric_columns = []
# for col in gdf.columns:
#         if pd.api.types.is_numeric_dtype(gdf[col]):
#             numeric_columns.append(col)
# 
#         elif pd.api.types.is_string_dtype(gdf[col]) and gdf[col].str.isnumeric().all():
#             gdf[col] = pd.to_numeric(gdf[col])  # Convert strings to numeric if all are numbers
#             numeric_columns.append(col)

# for col in gdf.columns:
#         gdf[col] = pd.to_numeric(gdf[col], errors='ignore')    

# numeric_columns = gdf.select_dtypes(include=[np.number]).columns.tolist()


for col in gdf.columns:
    try:
        pd.to_numeric(gdf[col])  # Attempt to convert to numeric
        numeric_columns.append(col)
    except ValueError:
        pass  # Ignore col
    except TypeError:
        pass
    except:
        pass
    
print(numeric_columns)

for col in numeric_columns:
    # gdf[col] = pd.to_numeric(gdf[col], errors='coerce').fillna(0)
    # gdf[col] = gdf[col].replace(None, 0)
    gdf[col] = pd.to_numeric(gdf[col], errors='coerce') 
    # Replace both NaN (from conversion) and existing None with 0
    gdf[col] = gdf[col].fillna(0)
    
# Basic information about the dataset
print("Dataset Info:")
gdf.info()



['Telephone_Number', 'UPRN', 'Latitude', 'Longitude']
Dataset Info:
<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 33 entries, 0 to 32
Data columns (total 19 columns):
 #   Column            Non-Null Count  Dtype   
---  ------            --------------  -----   
 0   Location_ID       33 non-null     object  
 1   HSCA_start_date   33 non-null     object  
 2   Name              33 non-null     object  
 3   Telephone_Number  33 non-null     float64 
 4   Web_Address       7 non-null      object  
 5   Type_Sector       33 non-null     object  
 6   Directorate       33 non-null     object  
 7   Category          33 non-null     object  
 8   Local_Authority   33 non-null     object  
 9   Street_Address    33 non-null     object  
 10  Address_Line_2    15 non-null     object  
 11  City              33 non-null     object  
 12  County            27 non-null     object  
 13  Postal_Code       33 non-null     object  
 14  UPRN              33 non-null     float64 
 15  

In [83]:
gdf

,Location_ID,HSCA_start_date,Name,Telephone_Number,Web_Address,Type_Sector,Directorate,Category,Local_Authority,Street_Address,Address_Line_2,City,County,Postal_Code,UPRN,Latitude,Longitude,Constituency,geometry
0,1-11088419348,09/08/2021,Whitehill House Dental Practice,1.422241e+09,None,Primary Dental Care,Primary medical services,Dentists,Calderdale,Whitehill Road,Illingworth,Halifax,West Yorkshire,HX2 9HD,1.000520e+11,53.751865,-1.891881,Halifax,POINT (-1.89188 53.75186)
1,1-12068961815,05/05/2022,J G Pearson Dental Surgeons - Southgate,1.422373e+09,None,Primary Dental Care,Primary medical services,Dentists,Calderdale,98 Southgate,None,Elland,West Yorkshire,HX5 0EP,1.000520e+11,53.684505,-1.836336,Calder Valley,POINT (-1.83634 53.6845)
2,1-12371306351,12/01/2022,K S Hunjan & Associates,1.422647e+09,www.hunjandental.com,Primary Dental Care,Primary medical services,Dentists,Calderdale,3 Parkinson Lane,None,Halifax,West Yorkshire,HX1 3XA,0.000000e+00,53.718520,-1.874676,Halifax,POINT (-1.87468 53.71852)
3,1-13134174655,18/07/2022,Carr Wood House Dental Surgery,0.000000e+00,None,Primary Dental Care,Primary medical services,Dentists,Calderdale,68 Carr House Road,Shelf,Halifax,None,HX3 7RJ,2.000020e+11,53.757230,-1.805794,Halifax,POINT (-1.80579 53.75723)
4,1-1315873667,30/05/2014,West Park Practice,1.484711e+09,None,Primary Dental Care,Primary medical services,Dentists,Calderdale,Unit 1,42 Park Street,Brighouse,West Yorkshire,HD6 1JL,1.000520e+11,53.701575,-1.780575,Calder Valley,POINT (-1.78058 53.70158)
5,1-13192244632,01/07/2022,Dental Care Centre Elland,1.422373e+09,www.ellandsmiles.co.uk,Primary Dental Care,Primary medical services,Dentists,Calderdale,98 Southgate,None,Elland,West Yorkshire,HX5 0EP,1.000520e+11,53.684505,-1.836336,Calder Valley,POINT (-1.83634 53.6845)
6,1-14444405398,01/12/2022,Bailiff Bridge Dental Practice,1.484717e+09,None,Primary Dental Care,Primary medical services,Dentists,Calderdale,629-631,Bradford Road,Brighouse,None,HD6 4DN,1.000520e+11,53.722001,-1.777314,Calder Valley,POINT (-1.77731 53.722)
7,1-1610731351,08/09/2014,Fresh Smiles Clinic,0.000000e+00,None,Primary Dental Care,Primary medical services,Dentists,Calderdale,2 High Street,None,Brighouse,West Yorkshire,HD6 1DE,0.000000e+00,53.703842,-1.780636,Calder Valley,POINT (-1.78064 53.70384)
8,1-1670753993,13/10/2014,West Vale Dental Surgery,1.422370e+09,None,Primary Dental Care,Primary medical services,Dentists,Calderdale,64 Saddleworth Road,West Vale,Halifax,West Yorkshire,HX4 8AG,1.000674e+10,53.686554,-1.855057,Calder Valley,POINT (-1.85506 53.68655)
9,1-1695451654,05/10/2014,Hebden Bridge Dental Care,1.422843e+09,None,Primary Dental Care,Primary medical services,Dentists,Calderdale,16 New Road,None,Hebden Bridge,West Yorkshire,HX7 8AD,1.000520e+11,53.740126,-2.010182,Calder Valley,POINT (-2.01018 53.74013)


In [84]:
identity_columns = ["ZipCode"]
drop = ["OBJECTID"]
target = []

In [85]:
# Summary statistics of numerical columns
print("\nSummary Statistics:")
gdf.describe()




Summary Statistics:


,Telephone_Number,UPRN,Latitude,Longitude
count,3.300000e+01,3.300000e+01,33.000000,33.000000
mean,1.182013e+09,6.337803e+10,53.713923,-1.879994
std,5.716497e+08,5.260260e+10,0.020427,0.088915
min,0.000000e+00,0.000000e+00,53.673682,-2.098748
25%,1.422241e+09,1.003504e+10,53.701575,-1.898432
50%,1.422365e+09,1.000520e+11,53.713077,-1.861054
75%,1.422834e+09,1.000520e+11,53.725116,-1.836336
max,1.706812e+09,2.000020e+11,53.757230,-1.777314


In [87]:
import geopandas as gpd
import pandas as pd
import numpy as np
from libpysal.weights import Queen
from esda.moran import Moran

def analyze_geojson(geojson_path, key_column):
    # Load the GeoJSON data into a GeoDataFrame
    gdf = gpd.read_file(geojson_path)

    # Print columns for debugging
    print("Columns in the GeoDataFrame:")
    print(gdf.columns)
    print(f"Key column: {key_column}")

    # Check if the key column exists in the GeoDataFrame
    if key_column not in gdf.columns:
        raise ValueError(f"Column '{key_column}' not found in the GeoDataFrame.")

    # Set the key column as the index if it's not already
    if gdf.index.name != key_column:
        gdf = gdf.set_index(key_column)

    # Separate geometry column if it exists
    if 'geometry' in gdf.columns:
        geometry = gdf['geometry']
        gdf = gdf.drop(columns=['geometry'])
    else:
        geometry = None
        print("Warning: No 'geometry' column found in the GeoDataFrame.")
    # 
    # if 'geometry' in gdf.columns:
    #     geometry = gdf['geometry'].copy()  # Create a copy of the geometry column
    #     gdf = gdf.drop(columns=['geometry']) 
    # else:
    #     geometry = None
    #     print("Warning: No 'geometry' column found in the GeoDataFrame.")

    # Identify numeric columns
    numeric_columns = []
    for col in gdf.columns:
        try:
            pd.to_numeric(gdf[col])  # Attempt to convert to numeric
            numeric_columns.append(col)
        except ValueError:
            pass  # Ignore col
        except TypeError:
            pass
        except:
            pass
        
    print(numeric_columns)
    
    for col in numeric_columns:
        # gdf[col] = pd.to_numeric(gdf[col], errors='coerce').fillna(0)
        # gdf[col] = gdf[col].replace(None, 0)
        gdf[col] = pd.to_numeric(gdf[col], errors='coerce') 
        # Replace both NaN (from conversion) and existing None with 0
        gdf[col] = gdf[col].fillna(0)
    # for col in gdf.columns:
    #     if pd.api.types.is_numeric_dtype(gdf[col]):
    #         numeric_columns.append(col)
    #         
    #     elif pd.api.types.is_string_dtype(gdf[col]) and gdf[col].str.isnumeric().all():
    #         gdf[col] = pd.to_numeric(gdf[col])  # Convert strings to numeric if all are numbers
    #         numeric_columns.append(col)
    # print(numeric_columns)
    
    
            

    if not numeric_columns:
        raise ValueError("No numeric columns found in the GeoDataFrame.")

    # Perform basic analysis
    analysis = {}

    # Statistics for numeric columns
    for stat in ['max', 'min', 'mean', 'sum', 'median', 'std']:
        analysis[stat] = getattr(gdf, stat)()[numeric_columns]

    # Count of all columns
    analysis['count'] = gdf.count()

    # Additional analyses
    analysis['unique_counts'] = gdf.nunique()

    # Find the key (e.g., ZipCode) with highest and lowest values for each numeric column
    for col in numeric_columns:
        max_key = gdf[col].idxmax()
        min_key = gdf[col].idxmin()
        analysis[f'{col}_highest'] = f"{key_column}: {max_key} value: ({gdf[col].max()})"
        analysis[f'{col}_lowest'] = f"{key_column}: {min_key} value: ({gdf[col].min()})"
    # 1. Correlation Matrix for Numeric Columns:
    correlation_matrix = gdf[numeric_columns].corr()
    analysis['correlation_matrix'] = correlation_matrix.to_dict()  # Convert to dict for easier handling by LLM

    # 2. Distribution Analysis (Skewness and Kurtosis):
    analysis['skewness'] = gdf[numeric_columns].skew().to_dict()
    analysis['kurtosis'] = gdf[numeric_columns].kurtosis().to_dict()

    # 3. Outlier Detection (using IQR):
    for col in numeric_columns:
        Q1 = gdf[col].quantile(0.25)
        Q3 = gdf[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        outliers = gdf[(gdf[col] < lower_bound) | (gdf[col] > upper_bound)].index.tolist()
        analysis[f'{col}_outliers'] = outliers 

    
    if geometry is not None:
        # Example using Moran's I (you might need to adjust the weights matrix)
        # Use the geometry copy directly in Queen.from_dataframe()
        w = Queen.from_dataframe(gpd.GeoDataFrame(geometry=geometry))
        for col in numeric_columns:
            mi = Moran(gdf[col], w)
            analysis[f'{col}_morans_i'] = mi.I  # Moran's I value

    return analysis, geometry

def print_analysis(analysis_results, key_column):
    for stat, result in analysis_results.items():
        if isinstance(result, pd.Series):
            print(f"{stat.capitalize()} values based on '{key_column}':")
            print(result.to_string())
        else:
            print(f"{stat}:")
            print(result)
        print("\n")

def main():
    geojson_path = "Dentists.geojson"
    key_column = 'Postal_Code'

    # try:
    analysis_results, geometry = analyze_geojson(geojson_path, key_column)
    print_analysis(analysis_results, key_column)

    # Optional: You can use the geometry for further spatial analysis if needed
    if geometry is not None:
        print(f"Geometry column is available for spatial analysis. Shape: {geometry.shape}")
    else:
        print("No geometry column found in the GeoDataFrame.")

    # except Exception as e:
    #     print(f"An error occurred: {str(e)}")

if __name__ == "__main__":
    main()






Columns in the GeoDataFrame:
Index(['Location_ID', 'HSCA_start_date', 'Name', 'Telephone_Number',
       'Web_Address', 'Type_Sector', 'Directorate', 'Category',
       'Local_Authority', 'Street_Address', 'Address_Line_2', 'City', 'County',
       'Postal_Code', 'UPRN', 'Latitude', 'Longitude', 'Constituency',
       'geometry'],
      dtype='object')
Key column: Postal_Code
['Telephone_Number', 'UPRN', 'Latitude', 'Longitude']


TypeError: '>=' not supported between instances of 'float' and 'str'

In [94]:
import geopandas as gpd
import pandas as pd
import numpy as np
from libpysal.weights import Queen
from esda.moran import Moran

def analyze_geojson(geojson_path, key_column):
    # Load the GeoJSON data into a GeoDataFrame
    gdf = gpd.read_file(geojson_path)

    print("Columns in the GeoDataFrame:")
    print(gdf.columns)
    print(f"Key column: {key_column}")

    if key_column not in gdf.columns:
        raise ValueError(f"Column '{key_column}' not found in the GeoDataFrame.")

    # Set the key column as the index if it's not already
    if gdf.index.name != key_column:
        gdf = gdf.set_index(key_column)

    # Separate geometry column if it exists
    if 'geometry' in gdf.columns:
        geometry = gdf['geometry']
        gdf = gdf.drop(columns=['geometry'])
    else:
        geometry = None
        print("Warning: No 'geometry' column found in the GeoDataFrame.")

    # Identify numeric columns
    numeric_columns = []
    for col in gdf.columns:
        try:
            pd.to_numeric(gdf[col])  # Attempt to convert to numeric
            numeric_columns.append(col)
        except ValueError:
            pass  # Ignore col
        except TypeError:
            pass
        except:
            pass
        
    print(numeric_columns)
    
    for col in numeric_columns:
        gdf[col] = pd.to_numeric(gdf[col], errors='coerce')
        # Replace both NaN (from conversion) and existing None with 0
        gdf[col] = gdf[col].fillna(0)

    if not numeric_columns:
        raise ValueError("No numeric columns found in the GeoDataFrame.")

    # Perform basic analysis
    analysis = {}

    # Statistics for numeric columns
    for stat in ['max', 'min', 'mean', 'sum', 'median', 'std']:
        analysis[stat] = getattr(gdf[numeric_columns], stat)()

    # Count of all columns
    analysis['count'] = gdf.count()

    # Additional analyses
    analysis['unique_counts'] = gdf.nunique()

    # Find the key with highest and lowest values for each numeric column
    for col in numeric_columns:
        max_key = gdf[col].idxmax()
        min_key = gdf[col].idxmin()
        analysis[f'{col}_highest'] = f"{key_column}: {max_key} value: ({gdf[col].max()})"
        analysis[f'{col}_lowest'] = f"{key_column}: {min_key} value: ({gdf[col].min()})"

    # Correlation Matrix for Numeric Columns
    correlation_matrix = gdf[numeric_columns].corr()
    analysis['correlation_matrix'] = correlation_matrix.to_dict()

    # Distribution Analysis (Skewness and Kurtosis)
    analysis['skewness'] = gdf[numeric_columns].skew().to_dict()
    analysis['kurtosis'] = gdf[numeric_columns].kurtosis().to_dict()

    # Outlier Detection (using IQR)
    for col in numeric_columns:
        Q1 = gdf[col].quantile(0.25)
        Q3 = gdf[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        outliers = gdf[(gdf[col] < lower_bound) | (gdf[col] > upper_bound)].index.tolist()
        analysis[f'{col}_outliers'] = outliers 

    if geometry is not None:
        # Spatial autocorrelation using Moran's I
        gdf_with_geometry = gpd.GeoDataFrame(gdf, geometry=geometry)
        w = Queen.from_dataframe(gdf_with_geometry)
        for col in numeric_columns:
            mi = Moran(gdf[col], w)
            analysis[f'{col}_morans_i'] = mi.I

    return analysis, geometry

def print_analysis(analysis_results, key_column):
    for stat, result in analysis_results.items():
        if isinstance(result, pd.Series):
            print(f"{stat.capitalize()} values based on '{key_column}':")
            print(result.to_string())
        else:
            print(f"{stat}:")
            print(result)
        print("\n")

def main():
    geojson_path = "data.geojson"
    key_column = 'Postal_Code'
    key_column = 'ZipCode'

    # try:
    analysis_results, geometry = analyze_geojson(geojson_path, key_column)
    print_analysis(analysis_results, key_column)

    # Optional: You can use the geometry for further spatial analysis if needed
    if geometry is not None:
        print(f"Geometry column is available for spatial analysis. Shape: {geometry.shape}")
    else:
        print("No geometry column found in the GeoDataFrame.")

    # except Exception as e:
    #     print(f"An error occurred: {str(e)}")

if __name__ == "__main__":
    main()


Columns in the GeoDataFrame:
Index(['OBJECTID', 'ZipCode', 'Total_population', 'Male_Total_Population',
       'Female_Total_Population', 'Percent_Male_Total_population__',
       'Percent_Female_Total_population', 'Total_population_AGE_Under_5_ye',
       'Total_population_AGE_5_to_9_yea', 'Total_population_AGE_10_to_14_y',
       ...
       'Percent_Female_Total_populat_26', 'Percent_Female_Total_populat_27',
       'Percent_Female_Total_populat_28', 'Percent_Female_Total_populat_29',
       'Percent_Female_Total_populat_30', 'Pop_by_Sq_Mile', 'SquareMiles',
       'Shape__Area', 'Shape__Length', 'geometry'],
      dtype='object', length=201)
Key column: ZipCode
['OBJECTID', 'Total_population', 'Male_Total_Population', 'Female_Total_Population', 'Percent_Male_Total_population__', 'Percent_Female_Total_population', 'Total_population_AGE_Under_5_ye', 'Total_population_AGE_5_to_9_yea', 'Total_population_AGE_10_to_14_y', 'Total_population_AGE_15_to_19_y', 'Total_population_AGE_20_to_24_y

C:\Users\The Just\AppData\Local\Temp\ipykernel_18560\3349593404.py:94: FutureWarning: `use_index` defaults to False but will default to True in future. Set True/False directly to control this behavior and silence this warning
  w = Queen.from_dataframe(gdf_with_geometry)


Max values based on 'ZipCode':
OBJECTID                           4.000000e+00
Total_population                   6.862300e+04
Male_Total_Population              3.907900e+04
Female_Total_Population            2.954400e+04
Percent_Male_Total_population__    5.694738e+01
Percent_Female_Total_population    5.148779e+01
Total_population_AGE_Under_5_ye    2.652000e+03
Total_population_AGE_5_to_9_yea    2.452000e+03
Total_population_AGE_10_to_14_y    2.349000e+03
Total_population_AGE_15_to_19_y    9.482000e+03
Total_population_AGE_20_to_24_y    1.839500e+04
Total_population_AGE_25_to_29_y    1.191900e+04
Total_population_AGE_30_to_34_y    5.939000e+03
Total_population_AGE_35_to_39_y    4.565000e+03
Total_population_AGE_40_to_44_y    3.116000e+03
Total_population_AGE_45_to_49_y    2.584000e+03
Total_population_AGE_50_to_54_y    2.680000e+03
Total_population_AGE_55_to_59_y    2.732000e+03
Total_population_AGE_60_to_64_y    2.993000e+03
Total_population_AGE_65_to_69_y    2.244000e+03
Total_pop